# Installations and Setup

In [ ]:
from google.colab import drive
import sys

In [ ]:
# TODO: Mount your Google Drive; this allows the runtime environment to access your drive.
drive.mount('/content/gdrive')

In [1]:
import IPython

ipython = IPython.get_ipython()
ipython.run_line_magic("load_ext", "autoreload")
ipython.run_line_magic("autoreload", "2")

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import lightning as L
from lightning.pytorch import loggers as pl_loggers

random.seed(0)
torch.manual_seed(0)

In [4]:
from models.architecture.assembled import AssembledModel
from models.train_loop import PatchTSTTrainer
from data.data_loader import TSDataLoader

In [5]:
patch_tst = AssembledModel()

# Train Model

In [6]:
output_dir = "./models/results"
logs_output_dir = "./models/results/logs"
model_trainer = PatchTSTTrainer(patch_tst, output_dir)
device = "cpu"
dataloader = TSDataLoader("./data/data_files/electricity/electricity.csv", device=device)
train_dataloader, val_dataloader = dataloader.get_data_loaders()

tb_logger = pl_loggers.TensorBoardLogger(save_dir=logs_output_dir)

trainer = L.Trainer(max_epochs=20, 
                    check_val_every_n_epoch=2,
                    num_sanity_val_steps=0, 
                    logger=tb_logger
                    )

/Users/jeffreyxiang/Desktop/HW/cs4782_s25_final_project/data/data_loader.py:30: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_stamp['hour'] = df_stamp.date.apply(lambda row: row.hour, 1)
/Users/jeffreyxiang/Desktop/HW/cs4782_s25_final_project/data/data_loader.py:31: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_stamp['day'] = df_stamp.date.apply(lambda row: row.day, 1)
/Users/jeffreyxiang/Desktop/HW/cs4782_s25_final_project/data/data_loader.py:32: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_stamp['month'] = df_stamp.date.apply(lambda row: row.month, 1)
/Users/jeffreyxiang/Desktop/

# Evaluate Model

In [8]:
print("Starting training...")
trainer.fit(model_trainer, train_dataloader, val_dataloader)
print("Training completed.")


  | Name  | Type           | Params | Mode 
-------------------------------------------------
0 | model | AssembledModel | 42.6 M | train
1 | loss  | MSELoss        | 0      | train
-------------------------------------------------
42.6 M    Trainable params
0         Non-trainable params
42.6 M    Total params
170.290   Total estimated model params size (MB)
168       Modules in train mode
0         Modules in eval mode


Starting training...
Epoch 0:   0%|          | 0/656 [00:00<?, ?it/s] x shape after normalization: torch.Size([32, 24, 321])
torch.Size([32, 1, 321])
(tensor([[[-1.4461,  0.9718, -0.8735,  ...,  0.5439, -1.7983, -0.0617]],

        [[-0.6018, -0.0219, -0.0350,  ..., -0.0488, -1.1463, -0.5188]],

        [[-0.6062,  1.9849,  0.0464,  ..., -0.5073,  0.8756,  0.6403]],

        ...,

        [[ 4.3197, -1.4264, -0.6736,  ..., -0.2458, -1.4822, -1.5186]],

        [[-1.6080, -0.5859, -1.0111,  ..., -3.6641, -2.3621, -1.8269]],

        [[ 0.7003, -0.3149,  6.1392,  ...,  0.0887, -0.6270,  0.3663]]]), tensor([[[-1.6970,  0.3312, -1.1862,  ..., -0.7497, -1.9525, -0.6519]],

        [[-0.9959, -0.0381, -0.4581,  ..., -0.7611, -1.4107, -0.7451]],

        [[-0.8927,  2.1477, -0.2609,  ..., -0.3429, -0.1801,  0.4732]],

        ...,

        [[-0.1247, -0.9890, -0.8188,  ..., -1.9729, -1.8319, -1.4525]],

        [[-1.4348, -0.7885, -1.0859,  ..., -2.9304, -2.0421, -1.6107]],

        [[ 3.2787

AttributeError: 'tuple' object has no attribute 'shape'